# Comparison of Time Series and Non-Time Series Model Performance
- Time Series : `RNN`, `GRU`, `LSTM`
- Non-Time Series : `Decision Tree`, `KNeighborsClassifier`, `BernoulliNB`, `MLPClassifier`, `AdaBoostClassifier`, `RandomForestClassifier`, `GradientBoostingClassifier`, `xgboost`, `lightgbm`, `catboost`

# Library

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN, GRU
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# Time Series Models
- **X : (7727, 10, 3595)**
- **Y : (7727, 1)**

In [2]:
x = np.load('x_(7727,10,3595).npy')

# LSTM
def LSTM_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return lstm
    
# RNN
def RNN_model():
    rnn = Sequential()
    rnn.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    rnn.add(SimpleRNN(units=128, activation='hard_sigmoid', return_sequences=True))
    rnn.add(SimpleRNN(units=64, activation='hard_sigmoid', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='hard_sigmoid', return_sequences=True))
    rnn.add(SimpleRNN(units=32, activation='hard_sigmoid', return_sequences=False))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=1, activation='sigmoid'))
    rnn.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return rnn
    
# GRU
def GRU_model():
    gru = Sequential()
    gru.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    gru.add(GRU(units=128, activation='hard_sigmoid', return_sequences=True))
    gru.add(GRU(units=64, activation='hard_sigmoid', return_sequences=True))
    gru.add(Dropout(0.2))
    gru.add(GRU(units=64, activation='hard_sigmoid', return_sequences=True))
    gru.add(GRU(units=32, activation='hard_sigmoid', return_sequences=False))
    gru.add(Dropout(0.2))
    gru.add(Dense(units=1, activation='sigmoid'))
    gru.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return gru

In [6]:
# Function for Training and Predicting Based on the Specified Model
def predict(model_name: str):
    
    path = '/project/LSH/** 해외_Journal of Biomedical Informatics/'
    with tf.device('/device:GPU:0'):

        # 1. Data load
        x = np.load(path + 'x_(7727,10,3595).npy')
        y = np.load(path + 'y_(7727,1).npy')

        acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []
        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

        # 2. Crossvalidation
        result = []
        for idx, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            print(f'{idx+1}')
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]

            # 3. Select Model (One of LSTM, RNN, GRU)
            if model_name == 'LSTM':
                model = LSTM_model()
            elif model_name == 'RNN':
                model = RNN_model()
            elif model_name == 'GRU':
                model = GRU_model()
            
            early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=False)
            model.fit(X_train, y_train, epochs=300, batch_size=516, validation_split=0.25, callbacks=[early_stop])

            y_pred_test = model.predict(X_test)
            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0

            acc = accuracy_score(y_test, y_pred_test)
            precision = precision_score(y_test, y_pred_test)
            recall = recall_score(y_test, y_pred_test)
            f1 = f1_score(y_test, y_pred_test)
            roc_auc = roc_auc_score(y_test, y_pred_test)
            
            result.append([model_name, acc, precision, recall, f1, roc_auc])
        
        df = pd.DataFrame(result)
        return df

## LSTM / RNN / GRU
- Using a 10-fold cross-validation method with a train to test ratio of 8:2 for predicting the survival/death of pneumonia patients.
- Among Acc, Precision, Recall, F1, and ROC-AUC, the ROC-AUC accuracy is used.


- [Explanation about the comparison criterion, ROC-AUC](https://bioinformaticsandme.tistory.com/328)
    - In clinical settings, the AUC-ROC curve is commonly used to evaluate the performance of a model in distinguishing between normal and patient classes.
    - The closer the AUC value is to 1, the better the model is at classifying classes.
    - The minimum value of AUC is 0.5, which means the model has no ability to differentiate between classes.

### Issue

In previous studies, the code was set to restore_best_weights = True, making LSTM perform better in terms of ROC-AUC than RNN and GRU, but for k-fold validation, shouldn't it be set to False? It's irrelevant since it's executed in parallel.

Results with restore_best_weights = False (based on 4068)
| Model | Acc | Precision | Recall | F1 | ROC-AUC |
|:-----:|:---:|:---------:|:------:|:--:|:-------:|
| LSTM | 0.747 | 0.799 | 0.783 | 0.791 | 0.737 |
| RNN | 0.753 | 0.794 | 0.804 | 0.799 | 0.738 |
| GRU | 0.749 | 0.797 | 0.792 | 0.794 | 0.737 |

### restore_best_weights = True

In [15]:
rnn_result = predict('RNN')
gru_result = predict('GRU')
lstm_result = predict('LSTM')

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 3s 117ms/step - loss: 0.6788 - acc: 0.5970 - val_loss: 0.6713 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 0s 48ms/step - loss: 0.6753 - acc: 0.5989 - val_loss: 0.6696 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6753 - acc: 0.6030 - val_loss: 0.6686 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 0s 48ms/step - loss: 0.6732 - acc: 0.5983 - val_loss: 0.6660 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6661 - acc: 0.6080 - val_loss: 0.6600 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6553 - acc: 0.6099 - val_loss: 0.6403 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6261 - acc: 0.6300 - val_loss: 0.6131 - val_acc: 0.6811
Epoch 8/300
9/9 [==============================] - 0s 48ms/step - loss: 0.5858 - acc:

Epoch 2/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6791 - acc: 0.5888 - val_loss: 0.6782 - val_acc: 0.5957
Epoch 3/300
9/9 [==============================] - 0s 50ms/step - loss: 0.6776 - acc: 0.6084 - val_loss: 0.6744 - val_acc: 0.5957
Epoch 4/300
9/9 [==============================] - 0s 48ms/step - loss: 0.6760 - acc: 0.5983 - val_loss: 0.6725 - val_acc: 0.5957
Epoch 5/300
9/9 [==============================] - 0s 50ms/step - loss: 0.6679 - acc: 0.6084 - val_loss: 0.6708 - val_acc: 0.5957
Epoch 6/300
9/9 [==============================] - 0s 50ms/step - loss: 0.6629 - acc: 0.6149 - val_loss: 0.6631 - val_acc: 0.5957
Epoch 7/300
9/9 [==============================] - 0s 48ms/step - loss: 0.6519 - acc: 0.6222 - val_loss: 0.6429 - val_acc: 0.5957
Epoch 8/300
9/9 [==============================] - 0s 50ms/step - loss: 0.6217 - acc: 0.6449 - val_loss: 0.6157 - val_acc: 0.6721
Epoch 9/300
9/9 [==============================] - 0s 49ms/step - loss: 0.5891 - acc: 0.69

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 4s 124ms/step - loss: 0.6943 - acc: 0.5474 - val_loss: 0.6723 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6884 - acc: 0.6054 - val_loss: 0.6765 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6789 - acc: 0.6039 - val_loss: 0.6706 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6814 - acc: 0.5858 - val_loss: 0.6705 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6754 - acc: 0.5953 - val_loss: 0.6709 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6764 - acc: 0.6004 - val_loss: 0.6703 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6752 - acc: 0.6035 - val_loss: 0.6694 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6738 - acc:

...

9/9 [==============================] - 1s 78ms/step - loss: 0.2795 - acc: 0.9016 - val_loss: 0.5919 - val_acc: 0.7445
Epoch 22/300
9/9 [==============================] - 1s 77ms/step - loss: 0.2672 - acc: 0.9059 - val_loss: 0.6107 - val_acc: 0.7490
Epoch 23/300
9/9 [==============================] - 1s 77ms/step - loss: 0.2687 - acc: 0.9031 - val_loss: 0.6271 - val_acc: 0.7445
Epoch 24/300
9/9 [==============================] - 1s 77ms/step - loss: 0.2525 - acc: 0.9092 - val_loss: 0.6293 - val_acc: 0.7503
Epoch 25/300
9/9 [==============================] - 1s 77ms/step - loss: 0.2355 - acc: 0.9219 - val_loss: 0.6874 - val_acc: 0.7529
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
[GRU] 정확도 : 0.763, Precision : 0.804, Recall : 0.810, F1 : 0.807, roc_auc : 0.750


0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 4s 126ms/step - loss: 0.6707 - acc: 0.6086 - val_loss: 0.6708 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6707 - acc: 0.6129 - val_loss: 0.6707 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6718 - acc: 0.6101 - val_loss: 0.6707 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6702 - acc: 0.6117 - val_loss: 0.6701 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6711 - acc: 0.6108 - val_loss: 0.6685 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6655 - acc: 0.6119 - val_loss: 0.6618 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 1s 76ms/step - loss: 0.6549 - acc: 0.6134 - val_loss: 0.6430 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6213 - acc:

9/9 [==============================] - 1s 79ms/step - loss: 0.6317 - acc: 0.6257 - val_loss: 0.6360 - val_acc: 0.5796
Epoch 12/300
9/9 [==============================] - 1s 79ms/step - loss: 0.5906 - acc: 0.6595 - val_loss: 0.5875 - val_acc: 0.7005
Epoch 13/300
9/9 [==============================] - 1s 78ms/step - loss: 0.5319 - acc: 0.7446 - val_loss: 0.5705 - val_acc: 0.7309
Epoch 14/300
9/9 [==============================] - 1s 78ms/step - loss: 0.4824 - acc: 0.7961 - val_loss: 0.5320 - val_acc: 0.7510
Epoch 15/300
9/9 [==============================] - 1s 79ms/step - loss: 0.4322 - acc: 0.8196 - val_loss: 0.5277 - val_acc: 0.7652
Epoch 16/300
9/9 [==============================] - 1s 80ms/step - loss: 0.3972 - acc: 0.8472 - val_loss: 0.5325 - val_acc: 0.7574
Epoch 17/300
9/9 [==============================] - 1s 79ms/step - loss: 0.3740 - acc: 0.8578 - val_loss: 0.5749 - val_acc: 0.7451
Epoch 18/300
9/9 [==============================] - 1s 79ms/step - loss: 0.3531 - acc: 0.8686 - 

9/9 [==============================] - 1s 79ms/step - loss: 0.5913 - acc: 0.6718 - val_loss: 0.5689 - val_acc: 0.7335
Epoch 15/300
9/9 [==============================] - 1s 78ms/step - loss: 0.5383 - acc: 0.7638 - val_loss: 0.5295 - val_acc: 0.7594
Epoch 16/300
9/9 [==============================] - 1s 78ms/step - loss: 0.4829 - acc: 0.8026 - val_loss: 0.5211 - val_acc: 0.7497
Epoch 17/300
9/9 [==============================] - 1s 79ms/step - loss: 0.4426 - acc: 0.8194 - val_loss: 0.5254 - val_acc: 0.7665
Epoch 18/300
9/9 [==============================] - 1s 80ms/step - loss: 0.4106 - acc: 0.8414 - val_loss: 0.5250 - val_acc: 0.7581
Epoch 19/300
9/9 [==============================] - 1s 80ms/step - loss: 0.3910 - acc: 0.8533 - val_loss: 0.5430 - val_acc: 0.7607
Epoch 20/300
9/9 [==============================] - 1s 77ms/step - loss: 0.3773 - acc: 0.8585 - val_loss: 0.5301 - val_acc: 0.7523
Epoch 21/300
9/9 [==============================] - 1s 80ms/step - loss: 0.3492 - acc: 0.8796 - 

In [34]:
# k-fold 10 df
result_df = pd.concat([lstm_result, rnn_result, gru_result])
result_df.columns = ['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']
result_df

# mean_df
LSTM_mean = result_df[result_df['Model']=='LSTM'].mean(axis=0)
RNN_mean = result_df[result_df['Model']=='RNN'].mean(axis=0)
GRU_mean = result_df[result_df['Model']=='GRU'].mean(axis=0)
mean_df = pd.DataFrame([LSTM_mean, RNN_mean, GRU_mean])
mean_df['Model'] = ['LSTM', 'RNN', 'GRU']
mean_df = mean_df[['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']]
mean_df

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,LSTM,0.757439,0.824401,0.765890,0.794069,0.755038
1,LSTM,0.765201,0.788481,0.841102,0.813942,0.743641
2,LSTM,0.756145,0.809836,0.784958,0.797203,0.747961
3,LSTM,0.758085,0.804487,0.797669,0.801064,0.746841
4,LSTM,0.761966,0.795688,0.820975,0.808133,0.745205
5,LSTM,0.771022,0.804124,0.826271,0.815047,0.755328
6,LSTM,0.755498,0.804301,0.792373,0.798292,0.745024
7,LSTM,0.757439,0.796663,0.809322,0.802943,0.742701
8,LSTM,0.743208,0.851091,0.702331,0.769588,0.754820
9,LSTM,0.738680,0.830073,0.719280,0.770715,0.744191


,Model,Acc,Precision,Recall,F1,ROC-AUC
0,LSTM,0.756468,0.810915,0.786017,0.797100,0.748075
1,RNN,0.761061,0.797515,0.816525,0.806581,0.745306
2,GRU,0.763195,0.804208,0.809958,0.806757,0.749912


### restore_best_weights = False

In [7]:
rnn_result = predict('RNN')
gru_result = predict('GRU')
lstm_result = predict('LSTM')

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 3s 92ms/step - loss: 0.6788 - acc: 0.6047 - val_loss: 0.6716 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 0s 48ms/step - loss: 0.6843 - acc: 0.5659 - val_loss: 0.6702 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 0s 52ms/step - loss: 0.6777 - acc: 0.6078 - val_loss: 0.6713 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 0s 51ms/step - loss: 0.6730 - acc: 0.6050 - val_loss: 0.6686 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 0s 50ms/step - loss: 0.6716 - acc: 0.6004 - val_loss: 0.6658 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6635 - acc: 0.6088 - val_loss: 0.6576 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 0s 50ms/step - loss: 0.6486 - acc: 0.6160 - val_loss: 0.6374 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6209 - acc: 

9/9 [==============================] - 0s 50ms/step - loss: 0.6717 - acc: 0.6039 - val_loss: 0.6680 - val_acc: 0.5957
Epoch 7/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6631 - acc: 0.6093 - val_loss: 0.6589 - val_acc: 0.5957
Epoch 8/300
9/9 [==============================] - 0s 49ms/step - loss: 0.6483 - acc: 0.6227 - val_loss: 0.6388 - val_acc: 0.5957
Epoch 9/300
9/9 [==============================] - 0s 50ms/step - loss: 0.6211 - acc: 0.6449 - val_loss: 0.6107 - val_acc: 0.6785
Epoch 10/300
9/9 [==============================] - 0s 50ms/step - loss: 0.5829 - acc: 0.6941 - val_loss: 0.5797 - val_acc: 0.6999
Epoch 11/300
9/9 [==============================] - 0s 50ms/step - loss: 0.5388 - acc: 0.7376 - val_loss: 0.5457 - val_acc: 0.7251
Epoch 12/300
9/9 [==============================] - 0s 50ms/step - loss: 0.4899 - acc: 0.7748 - val_loss: 0.5212 - val_acc: 0.7380
Epoch 13/300
9/9 [==============================] - 0s 49ms/step - loss: 0.4515 - acc: 0.7978 - val

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 4s 154ms/step - loss: 0.8191 - acc: 0.4041 - val_loss: 0.7121 - val_acc: 0.3946
Epoch 2/300
9/9 [==============================] - 1s 77ms/step - loss: 0.6908 - acc: 0.5515 - val_loss: 0.6731 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6821 - acc: 0.6104 - val_loss: 0.6791 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6807 - acc: 0.6056 - val_loss: 0.6706 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6783 - acc: 0.5968 - val_loss: 0.6698 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6766 - acc: 0.5922 - val_loss: 0.6691 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6729 - acc: 0.6000 - val_loss: 0.6685 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6730 - acc:

...

9/9 [==============================] - 1s 78ms/step - loss: 0.2489 - acc: 0.9148 - val_loss: 0.6521 - val_acc: 0.7555
Epoch 00023: early stopping
9번째
Epoch 1/300
9/9 [==============================] - 4s 126ms/step - loss: 0.7435 - acc: 0.4548 - val_loss: 0.6745 - val_acc: 0.6229
Epoch 2/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6829 - acc: 0.5883 - val_loss: 0.6653 - val_acc: 0.6229
Epoch 3/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6877 - acc: 0.6019 - val_loss: 0.6628 - val_acc: 0.6229
Epoch 4/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6806 - acc: 0.5875 - val_loss: 0.6642 - val_acc: 0.6229
Epoch 5/300
9/9 [==============================] - 1s 76ms/step - loss: 0.6793 - acc: 0.5804 - val_loss: 0.6638 - val_acc: 0.6229
Epoch 6/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6772 - acc: 0.5953 - val_loss: 0.6617 - val_acc: 0.6229
Epoch 7/300
9/9 [==============================] - 1s 80ms/step - los

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 4s 129ms/step - loss: 0.6788 - acc: 0.5950 - val_loss: 0.6716 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 1s 80ms/step - loss: 0.6777 - acc: 0.6080 - val_loss: 0.6710 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6746 - acc: 0.6058 - val_loss: 0.6704 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 1s 80ms/step - loss: 0.6720 - acc: 0.6058 - val_loss: 0.6698 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 1s 80ms/step - loss: 0.6685 - acc: 0.6097 - val_loss: 0.6676 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6646 - acc: 0.6125 - val_loss: 0.6590 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 1s 80ms/step - loss: 0.6453 - acc: 0.6173 - val_loss: 0.6323 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6019 - acc:

9/9 [==============================] - 1s 79ms/step - loss: 0.5004 - acc: 0.7655 - val_loss: 0.5361 - val_acc: 0.7471
Epoch 11/300
9/9 [==============================] - 1s 79ms/step - loss: 0.4541 - acc: 0.7972 - val_loss: 0.5483 - val_acc: 0.7574
Epoch 12/300
9/9 [==============================] - 1s 80ms/step - loss: 0.4189 - acc: 0.8192 - val_loss: 0.5186 - val_acc: 0.7646
Epoch 13/300
9/9 [==============================] - 1s 79ms/step - loss: 0.3853 - acc: 0.8397 - val_loss: 0.5263 - val_acc: 0.7600
Epoch 14/300
9/9 [==============================] - 1s 80ms/step - loss: 0.3626 - acc: 0.8533 - val_loss: 0.5542 - val_acc: 0.7607
Epoch 15/300
9/9 [==============================] - 1s 80ms/step - loss: 0.3337 - acc: 0.8749 - val_loss: 0.5786 - val_acc: 0.7594
Epoch 16/300
9/9 [==============================] - 1s 81ms/step - loss: 0.3083 - acc: 0.8898 - val_loss: 0.5954 - val_acc: 0.7497
Epoch 17/300
9/9 [==============================] - 1s 80ms/step - loss: 0.2840 - acc: 0.9018 - 

9/9 [==============================] - 1s 78ms/step - loss: 0.6684 - acc: 0.6123 - val_loss: 0.6767 - val_acc: 0.5899
Epoch 4/300
9/9 [==============================] - 1s 81ms/step - loss: 0.6703 - acc: 0.6149 - val_loss: 0.6784 - val_acc: 0.5899
Epoch 5/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6671 - acc: 0.6157 - val_loss: 0.6789 - val_acc: 0.5899
Epoch 6/300
9/9 [==============================] - 1s 79ms/step - loss: 0.6658 - acc: 0.6170 - val_loss: 0.6761 - val_acc: 0.5899
Epoch 7/300
9/9 [==============================] - 1s 80ms/step - loss: 0.6655 - acc: 0.6147 - val_loss: 0.6733 - val_acc: 0.5899
Epoch 8/300
9/9 [==============================] - 1s 78ms/step - loss: 0.6580 - acc: 0.6173 - val_loss: 0.6633 - val_acc: 0.5899
Epoch 9/300
9/9 [==============================] - 1s 80ms/step - loss: 0.6420 - acc: 0.6192 - val_loss: 0.6352 - val_acc: 0.5899
Epoch 10/300
9/9 [==============================] - 1s 80ms/step - loss: 0.5967 - acc: 0.6522 - val_lo

In [5]:
# k-fold 10 df
result_df = pd.concat([lstm_result, rnn_result, gru_result])
result_df.columns = ['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']
result_df

# mean_df
LSTM_mean = result_df[result_df['Model']=='LSTM'].mean(axis=0)
RNN_mean = result_df[result_df['Model']=='RNN'].mean(axis=0)
GRU_mean = result_df[result_df['Model']=='GRU'].mean(axis=0)
mean_df = pd.DataFrame([LSTM_mean, RNN_mean, GRU_mean])
mean_df['Model'] = ['LSTM', 'RNN', 'GRU']
mean_df = mean_df[['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']]
mean_df

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,,NaN,NaN,NaN,NaN,NaN
1,LSTM,0.752911,0.802802,0.789195,0.795940,0.742604
2,LSTM,0.757439,0.802338,0.799788,0.801061,0.745409
3,LSTM,0.750323,0.796809,0.793432,0.795117,0.738078
4,LSTM,0.753558,0.832349,0.746822,0.787270,0.755471
5,LSTM,0.757439,0.771729,0.855932,0.811652,0.729461
6,LSTM,0.766494,0.811765,0.804025,0.807877,0.755833
7,LSTM,0.738034,0.829865,0.718220,0.770017,0.743662
8,LSTM,0.753558,0.799150,0.796610,0.797878,0.741328
9,LSTM,0.763907,0.785222,0.844280,0.813680,0.741077


,Model,Acc,Precision,Recall,F1,ROC-AUC
0,LSTM,0.752911,0.800527,0.795021,0.796780,0.740949
1,RNN,0.748836,0.796091,0.792267,0.793464,0.736499
2,GRU,0.753105,0.798520,0.797669,0.797568,0.740446


# Non-Time Series Models
- **X : (7727, 3595)**
- **Y : (7727, 1)**

In [8]:
def predict2(model):
    
    with tf.device('/device:GPU:0'):
        
        # 1. Data load
        x = np.load('x_(7727,3595).npy')
        y = np.load('y_(7727,1).npy')

        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)
        acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []

        # 2. Crossvalidation
        result = []
        for (train_index, test_index) in tqdm(sss.split(x, y)):
                    
            X_train, y_train = x[train_index,:], y[train_index]
            X_test, y_test = x[test_index,:], y[test_index]
            model.fit(X_train, y_train)
            y_pred_test = model.predict(X_test)
            
            acc = accuracy_score(y_test, y_pred_test)
            precision = precision_score(y_test, y_pred_test)
            recall = recall_score(y_test, y_pred_test)
            f1 = f1_score(y_test, y_pred_test)
            roc_auc = roc_auc_score(y_test, y_pred_test)
            
            result.append([str(model)[:-2], np.round(acc, 3), np.round(precision, 3), np.round(recall, 3), np.round(f1, 3), np.round(roc_auc, 3)])
        
        df = pd.DataFrame(result)
        df.columns = ['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']
        return df

## Decision Tree (DT)

In [42]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
DT = predict2(model)
DT
np.mean(DT['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,DecisionTreeClassifier,0.625,0.692,0.696,0.694,0.605
1,DecisionTreeClassifier,0.668,0.721,0.743,0.732,0.646
2,DecisionTreeClassifier,0.644,0.715,0.695,0.705,0.630
3,DecisionTreeClassifier,0.644,0.707,0.713,0.710,0.625
4,DecisionTreeClassifier,0.668,0.732,0.719,0.726,0.654
5,DecisionTreeClassifier,0.672,0.736,0.721,0.729,0.658
6,DecisionTreeClassifier,0.670,0.723,0.745,0.734,0.649
7,DecisionTreeClassifier,0.647,0.704,0.728,0.716,0.624
8,DecisionTreeClassifier,0.673,0.730,0.736,0.733,0.655
9,DecisionTreeClassifier,0.653,0.728,0.688,0.707,0.643


0.6388999999999999

## KNeighborsClassifier (KN)

In [43]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
KN = predict2(model)
KN
np.mean(KN['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,KNeighborsClassifier,0.659,0.663,0.898,0.763,0.591
1,KNeighborsClassifier,0.661,0.669,0.879,0.760,0.599
2,KNeighborsClassifier,0.653,0.662,0.882,0.756,0.587
3,KNeighborsClassifier,0.653,0.663,0.879,0.756,0.589
4,KNeighborsClassifier,0.651,0.656,0.899,0.759,0.580
5,KNeighborsClassifier,0.657,0.663,0.891,0.760,0.591
6,KNeighborsClassifier,0.669,0.674,0.888,0.766,0.607
7,KNeighborsClassifier,0.669,0.670,0.906,0.770,0.602
8,KNeighborsClassifier,0.664,0.663,0.915,0.769,0.592
9,KNeighborsClassifier,0.655,0.663,0.881,0.757,0.590


0.5927999999999999

## BernoulliNB (BNB)

In [44]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
BNB = predict2(model)
BNB
np.mean(BNB['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,BernoulliNB,0.685,0.774,0.684,0.726,0.685
1,BernoulliNB,0.692,0.779,0.692,0.733,0.692
2,BernoulliNB,0.669,0.767,0.658,0.708,0.672
3,BernoulliNB,0.664,0.755,0.664,0.707,0.663
4,BernoulliNB,0.684,0.769,0.689,0.727,0.682
5,BernoulliNB,0.691,0.773,0.699,0.734,0.688
6,BernoulliNB,0.673,0.768,0.665,0.713,0.675
7,BernoulliNB,0.670,0.747,0.696,0.720,0.663
8,BernoulliNB,0.695,0.773,0.709,0.739,0.691
9,BernoulliNB,0.695,0.786,0.688,0.733,0.697


0.6808

## MLPClassifier (MLP)

In [45]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
MLP = predict2(model)
MLP
np.mean(MLP['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,MLPClassifier,0.726,0.778,0.771,0.774,0.713
1,MLPClassifier,0.743,0.786,0.794,0.790,0.728
2,MLPClassifier,0.743,0.787,0.794,0.791,0.729
3,MLPClassifier,0.748,0.790,0.800,0.795,0.733
4,MLPClassifier,0.724,0.764,0.793,0.778,0.704
5,MLPClassifier,0.715,0.763,0.774,0.769,0.699
6,MLPClassifier,0.728,0.771,0.789,0.780,0.711
7,MLPClassifier,0.724,0.767,0.786,0.777,0.706
8,MLPClassifier,0.729,0.778,0.779,0.778,0.715
9,MLPClassifier,0.721,0.780,0.755,0.767,0.711


0.7148999999999999

## AdaBoostClassifier (AB)

In [46]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
AB = predict2(model)
AB
np.mean(AB['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,AdaBoostClassifier,0.746,0.780,0.815,0.797,0.727
1,AdaBoostClassifier,0.745,0.781,0.810,0.795,0.727
2,AdaBoostClassifier,0.731,0.775,0.788,0.782,0.715
3,AdaBoostClassifier,0.738,0.766,0.823,0.793,0.714
4,AdaBoostClassifier,0.737,0.775,0.803,0.789,0.719
5,AdaBoostClassifier,0.748,0.771,0.836,0.802,0.723
6,AdaBoostClassifier,0.746,0.777,0.819,0.797,0.725
7,AdaBoostClassifier,0.710,0.748,0.793,0.770,0.687
8,AdaBoostClassifier,0.732,0.769,0.803,0.785,0.712
9,AdaBoostClassifier,0.755,0.792,0.812,0.802,0.738


0.7186999999999999

## RandomForestClassifier (RF)

In [47]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
RF = predict2(model)
RF
np.mean(RF['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,RandomForestClassifier,0.750,0.749,0.889,0.813,0.711
1,RandomForestClassifier,0.750,0.754,0.876,0.810,0.714
2,RandomForestClassifier,0.737,0.753,0.847,0.798,0.706
3,RandomForestClassifier,0.739,0.745,0.871,0.803,0.701
4,RandomForestClassifier,0.741,0.747,0.872,0.804,0.704
5,RandomForestClassifier,0.743,0.745,0.880,0.807,0.703
6,RandomForestClassifier,0.750,0.754,0.877,0.811,0.714
7,RandomForestClassifier,0.741,0.748,0.870,0.804,0.705
8,RandomForestClassifier,0.748,0.746,0.891,0.812,0.708
9,RandomForestClassifier,0.748,0.752,0.877,0.810,0.712


0.7078

## GradientBoostingClassifier (GB)

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
GB = predict2(model)
GB
np.mean(GB['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,GradientBoostingClassifier,0.756,0.768,0.861,0.812,0.726
1,GradientBoostingClassifier,0.750,0.762,0.857,0.807,0.719
2,GradientBoostingClassifier,0.738,0.766,0.822,0.793,0.714
3,GradientBoostingClassifier,0.741,0.756,0.850,0.800,0.710
4,GradientBoostingClassifier,0.745,0.758,0.854,0.803,0.713
5,GradientBoostingClassifier,0.750,0.759,0.867,0.809,0.717
6,GradientBoostingClassifier,0.761,0.771,0.865,0.815,0.731
7,GradientBoostingClassifier,0.741,0.753,0.856,0.801,0.708
8,GradientBoostingClassifier,0.743,0.754,0.859,0.803,0.709
9,GradientBoostingClassifier,0.760,0.771,0.863,0.815,0.731


0.7177999999999999

## xgboost (XGB)

In [14]:
# pip install xgboost

In [13]:
from xgboost import XGBClassifier
model = XGBClassifier() 
XGB = predict2(model)
XGB
np.mean(XGB['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.763,0.786,0.839,0.812,0.741
1,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.756,0.789,0.820,0.804,0.738
2,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.767,0.800,0.824,0.812,0.751
3,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.757,0.781,0.837,0.808,0.735
4,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.746,0.776,0.822,0.798,0.725
5,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.774,0.794,0.851,0.821,0.753
6,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.763,0.784,0.845,0.813,0.740
7,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.759,0.777,0.849,0.811,0.733
8,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.755,0.781,0.834,0.806,0.733
9,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.748,0.780,0.819,0.799,0.728


0.7376999999999999

## lightgbm (LGBM)

In [29]:
import lightgbm

In [30]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
LGBM = predict2(model)
LGBM
np.mean(LGBM['ROC-AUC'])

0it [00:00, ?it/s]

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,LGBMClassifier,0.771,0.790,0.851,0.819,0.748
1,LGBMClassifier,0.757,0.786,0.827,0.806,0.737
2,LGBMClassifier,0.767,0.801,0.823,0.812,0.751
3,LGBMClassifier,0.761,0.787,0.835,0.810,0.740
4,LGBMClassifier,0.759,0.781,0.840,0.810,0.736
5,LGBMClassifier,0.781,0.798,0.860,0.828,0.759
6,LGBMClassifier,0.762,0.783,0.843,0.812,0.739
7,LGBMClassifier,0.756,0.780,0.837,0.807,0.733
8,LGBMClassifier,0.763,0.788,0.836,0.811,0.742
9,LGBMClassifier,0.774,0.798,0.843,0.820,0.754


0.7439

## catboost (CAT)

In [9]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
CAT = predict2(model)
CAT
np.mean(CAT['ROC-AUC'])

0it [00:00, ?it/s]

Learning rate set to 0.022424
0:	learn: 0.6871222	total: 59.4ms	remaining: 59.4s
1:	learn: 0.6827865	total: 70.4ms	remaining: 35.1s
2:	learn: 0.6794971	total: 81.1ms	remaining: 27s
3:	learn: 0.6755913	total: 92.1ms	remaining: 22.9s
4:	learn: 0.6713823	total: 103ms	remaining: 20.5s
5:	learn: 0.6667788	total: 114ms	remaining: 18.9s
6:	learn: 0.6637496	total: 125ms	remaining: 17.7s
7:	learn: 0.6598405	total: 136ms	remaining: 16.9s
8:	learn: 0.6559675	total: 147ms	remaining: 16.2s
9:	learn: 0.6530094	total: 159ms	remaining: 15.7s
10:	learn: 0.6489843	total: 170ms	remaining: 15.3s
11:	learn: 0.6460378	total: 181ms	remaining: 14.9s
12:	learn: 0.6423627	total: 193ms	remaining: 14.6s
13:	learn: 0.6386850	total: 204ms	remaining: 14.4s
14:	learn: 0.6359757	total: 216ms	remaining: 14.2s
15:	learn: 0.6332219	total: 227ms	remaining: 14s
16:	learn: 0.6305387	total: 239ms	remaining: 13.8s
17:	learn: 0.6286540	total: 250ms	remaining: 13.6s
18:	learn: 0.6265563	total: 261ms	remaining: 13.5s
19:	learn: 

175:	learn: 0.5099125	total: 2.03s	remaining: 9.52s
176:	learn: 0.5096278	total: 2.04s	remaining: 9.51s
177:	learn: 0.5092943	total: 2.06s	remaining: 9.49s
178:	learn: 0.5090418	total: 2.07s	remaining: 9.48s
179:	learn: 0.5088315	total: 2.08s	remaining: 9.47s
180:	learn: 0.5084886	total: 2.09s	remaining: 9.46s
181:	learn: 0.5082188	total: 2.1s	remaining: 9.44s
182:	learn: 0.5079500	total: 2.11s	remaining: 9.43s
183:	learn: 0.5076307	total: 2.12s	remaining: 9.42s
184:	learn: 0.5073250	total: 2.13s	remaining: 9.4s
185:	learn: 0.5069975	total: 2.15s	remaining: 9.39s
186:	learn: 0.5068142	total: 2.16s	remaining: 9.38s
187:	learn: 0.5065052	total: 2.17s	remaining: 9.36s
188:	learn: 0.5061644	total: 2.18s	remaining: 9.35s
189:	learn: 0.5058937	total: 2.19s	remaining: 9.33s
190:	learn: 0.5057077	total: 2.2s	remaining: 9.32s
191:	learn: 0.5053354	total: 2.21s	remaining: 9.3s
192:	learn: 0.5051254	total: 2.22s	remaining: 9.29s
193:	learn: 0.5047744	total: 2.23s	remaining: 9.27s
194:	learn: 0.50

339:	learn: 0.4718127	total: 3.87s	remaining: 7.52s
340:	learn: 0.4716614	total: 3.88s	remaining: 7.51s
341:	learn: 0.4714549	total: 3.9s	remaining: 7.5s
342:	learn: 0.4712863	total: 3.91s	remaining: 7.49s
343:	learn: 0.4710605	total: 3.92s	remaining: 7.47s
344:	learn: 0.4708958	total: 3.93s	remaining: 7.46s
345:	learn: 0.4706966	total: 3.94s	remaining: 7.45s
346:	learn: 0.4704084	total: 3.95s	remaining: 7.44s
347:	learn: 0.4700932	total: 3.96s	remaining: 7.43s
348:	learn: 0.4698325	total: 3.98s	remaining: 7.42s
349:	learn: 0.4696547	total: 3.99s	remaining: 7.41s
350:	learn: 0.4695099	total: 4s	remaining: 7.39s
351:	learn: 0.4693375	total: 4.01s	remaining: 7.38s
352:	learn: 0.4690617	total: 4.02s	remaining: 7.37s
353:	learn: 0.4688940	total: 4.03s	remaining: 7.36s
354:	learn: 0.4687011	total: 4.04s	remaining: 7.35s
355:	learn: 0.4684880	total: 4.06s	remaining: 7.34s
356:	learn: 0.4682389	total: 4.07s	remaining: 7.33s
357:	learn: 0.4680876	total: 4.08s	remaining: 7.31s
358:	learn: 0.467

501:	learn: 0.4344810	total: 5.71s	remaining: 5.67s
502:	learn: 0.4342977	total: 5.72s	remaining: 5.66s
503:	learn: 0.4341175	total: 5.74s	remaining: 5.64s
504:	learn: 0.4338213	total: 5.75s	remaining: 5.63s
505:	learn: 0.4335670	total: 5.76s	remaining: 5.62s
506:	learn: 0.4332716	total: 5.77s	remaining: 5.61s
507:	learn: 0.4330491	total: 5.78s	remaining: 5.6s
508:	learn: 0.4328309	total: 5.79s	remaining: 5.59s
509:	learn: 0.4325999	total: 5.8s	remaining: 5.58s
510:	learn: 0.4323534	total: 5.81s	remaining: 5.56s
511:	learn: 0.4321025	total: 5.83s	remaining: 5.55s
512:	learn: 0.4318486	total: 5.84s	remaining: 5.54s
513:	learn: 0.4316775	total: 5.85s	remaining: 5.53s
514:	learn: 0.4314341	total: 5.86s	remaining: 5.52s
515:	learn: 0.4311077	total: 5.87s	remaining: 5.51s
516:	learn: 0.4308837	total: 5.88s	remaining: 5.5s
517:	learn: 0.4306849	total: 5.89s	remaining: 5.48s
518:	learn: 0.4303556	total: 5.9s	remaining: 5.47s
519:	learn: 0.4300495	total: 5.92s	remaining: 5.46s
520:	learn: 0.42

...

645:	learn: 0.4048847	total: 7.49s	remaining: 4.11s
646:	learn: 0.4046970	total: 7.5s	remaining: 4.09s
647:	learn: 0.4044669	total: 7.51s	remaining: 4.08s
648:	learn: 0.4042596	total: 7.53s	remaining: 4.07s
649:	learn: 0.4040193	total: 7.54s	remaining: 4.06s
650:	learn: 0.4037609	total: 7.55s	remaining: 4.05s
651:	learn: 0.4036963	total: 7.56s	remaining: 4.04s
652:	learn: 0.4034603	total: 7.57s	remaining: 4.02s
653:	learn: 0.4031916	total: 7.58s	remaining: 4.01s
654:	learn: 0.4029595	total: 7.6s	remaining: 4s
655:	learn: 0.4028014	total: 7.61s	remaining: 3.99s
656:	learn: 0.4026507	total: 7.62s	remaining: 3.98s
657:	learn: 0.4024932	total: 7.63s	remaining: 3.97s
658:	learn: 0.4023382	total: 7.64s	remaining: 3.95s
659:	learn: 0.4022786	total: 7.65s	remaining: 3.94s
660:	learn: 0.4020357	total: 7.67s	remaining: 3.93s
661:	learn: 0.4018305	total: 7.68s	remaining: 3.92s
662:	learn: 0.4016551	total: 7.69s	remaining: 3.91s
663:	learn: 0.4014444	total: 7.7s	remaining: 3.9s
664:	learn: 0.40127

807:	learn: 0.3758689	total: 9.38s	remaining: 2.23s
808:	learn: 0.3758089	total: 9.39s	remaining: 2.22s
809:	learn: 0.3757058	total: 9.4s	remaining: 2.21s
810:	learn: 0.3755023	total: 9.42s	remaining: 2.19s
811:	learn: 0.3753384	total: 9.43s	remaining: 2.18s
812:	learn: 0.3751847	total: 9.44s	remaining: 2.17s
813:	learn: 0.3749551	total: 9.45s	remaining: 2.16s
814:	learn: 0.3748972	total: 9.46s	remaining: 2.15s
815:	learn: 0.3747042	total: 9.47s	remaining: 2.14s
816:	learn: 0.3745209	total: 9.49s	remaining: 2.12s
817:	learn: 0.3743114	total: 9.5s	remaining: 2.11s
818:	learn: 0.3741414	total: 9.51s	remaining: 2.1s
819:	learn: 0.3739296	total: 9.52s	remaining: 2.09s
820:	learn: 0.3736963	total: 9.53s	remaining: 2.08s
821:	learn: 0.3735357	total: 9.54s	remaining: 2.07s
822:	learn: 0.3733853	total: 9.55s	remaining: 2.06s
823:	learn: 0.3732569	total: 9.57s	remaining: 2.04s
824:	learn: 0.3730761	total: 9.58s	remaining: 2.03s
825:	learn: 0.3728990	total: 9.59s	remaining: 2.02s
826:	learn: 0.3

969:	learn: 0.3506525	total: 11.3s	remaining: 349ms
970:	learn: 0.3505312	total: 11.3s	remaining: 337ms
971:	learn: 0.3504063	total: 11.3s	remaining: 325ms
972:	learn: 0.3502623	total: 11.3s	remaining: 314ms
973:	learn: 0.3501716	total: 11.3s	remaining: 302ms
974:	learn: 0.3500438	total: 11.3s	remaining: 291ms
975:	learn: 0.3499239	total: 11.3s	remaining: 279ms
976:	learn: 0.3498983	total: 11.4s	remaining: 267ms
977:	learn: 0.3497347	total: 11.4s	remaining: 256ms
978:	learn: 0.3495135	total: 11.4s	remaining: 244ms
979:	learn: 0.3493722	total: 11.4s	remaining: 232ms
980:	learn: 0.3492103	total: 11.4s	remaining: 221ms
981:	learn: 0.3490529	total: 11.4s	remaining: 209ms
982:	learn: 0.3488539	total: 11.4s	remaining: 198ms
983:	learn: 0.3485924	total: 11.4s	remaining: 186ms
984:	learn: 0.3485643	total: 11.5s	remaining: 174ms
985:	learn: 0.3484209	total: 11.5s	remaining: 163ms
986:	learn: 0.3481958	total: 11.5s	remaining: 151ms
987:	learn: 0.3480394	total: 11.5s	remaining: 140ms
988:	learn: 

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,<catboost.core.CatBoostClassifier object at 0x...,0.757,0.776,0.847,0.810,0.732
1,<catboost.core.CatBoostClassifier object at 0x...,0.769,0.788,0.851,0.818,0.746
2,<catboost.core.CatBoostClassifier object at 0x...,0.758,0.786,0.829,0.807,0.738
3,<catboost.core.CatBoostClassifier object at 0x...,0.760,0.777,0.851,0.812,0.734
4,<catboost.core.CatBoostClassifier object at 0x...,0.763,0.780,0.852,0.814,0.737
5,<catboost.core.CatBoostClassifier object at 0x...,0.772,0.782,0.869,0.823,0.745
6,<catboost.core.CatBoostClassifier object at 0x...,0.781,0.792,0.869,0.829,0.756
7,<catboost.core.CatBoostClassifier object at 0x...,0.759,0.775,0.854,0.812,0.733
8,<catboost.core.CatBoostClassifier object at 0x...,0.760,0.779,0.847,0.812,0.735
9,<catboost.core.CatBoostClassifier object at 0x...,0.768,0.788,0.850,0.818,0.745


0.7401000000000001